In [ ]:
!pip install dash

In [5]:
import pandas as pd
import dash
import dash_html_components as html
import dash_core_components as dcc
from dash.dependencies import Input, Output
import plotly.express as px
spacex_df = pd.read_csv("D:\sagar\coursera peer assignment\course 10 capstone project\spacex_launch_dash.csv")
spacex_df.head()

<>:7: SyntaxWarning: invalid escape sequence '\s'
<>:7: SyntaxWarning: invalid escape sequence '\s'
C:\Users\sagar\AppData\Local\Temp\ipykernel_1524\752133265.py:7: SyntaxWarning: invalid escape sequence '\s'
  spacex_df = pd.read_csv("D:\sagar\coursera peer assignment\course 10 capstone project\spacex_launch_dash.csv")
C:\Users\sagar\AppData\Local\Temp\ipykernel_1524\752133265.py:3: UserWarning: 
The dash_html_components package is deprecated. Please replace
`import dash_html_components as html` with `from dash import html`
  import dash_html_components as html
C:\Users\sagar\AppData\Local\Temp\ipykernel_1524\752133265.py:4: UserWarning: 
The dash_core_components package is deprecated. Please replace
`import dash_core_components as dcc` with `from dash import dcc`
  import dash_core_components as dcc


,Unnamed: 0,Flight Number,Launch Site,class,Payload Mass (kg),Booster Version,Booster Version Category
0,0,1,CCAFS LC-40,0,0.0,F9 v1.0 B0003,v1.0
1,1,2,CCAFS LC-40,0,0.0,F9 v1.0 B0004,v1.0
2,2,3,CCAFS LC-40,0,525.0,F9 v1.0 B0005,v1.0
3,3,4,CCAFS LC-40,0,500.0,F9 v1.0 B0006,v1.0
4,4,5,CCAFS LC-40,0,677.0,F9 v1.0 B0007,v1.0


In [27]:
# Correct filtering and column selection
bar_filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= 0.0) & 
                            (spacex_df['Payload Mass (kg)'] <= 500.0)]
success_rate_df = bar_filtered_df[['class', 'Payload Mass (kg)']]
print(success_rate_df.head())


    class  Payload Mass (kg)
0       0                0.0
1       0                0.0
3       0              500.0
26      0              500.0
30      1              475.0


In [29]:
# Read the airline data into pandas dataframe
max_payload = spacex_df['Payload Mass (kg)'].max()
min_payload = spacex_df['Payload Mass (kg)'].min()

# Create a dash application
app = dash.Dash(__name__)

# Create an app layout
app.layout = html.Div(children=[
    html.H1('SpaceX Launch Records Dashboard',
            style={'textAlign': 'center', 'color': '#503D36', 'font-size': 40}),
    html.P("Select Launch Site:"),
    # TASK 1: Add a dropdown list to enable Launch Site selection
    # The default select value is for ALL sites
    dcc.Dropdown(id='site-dropdown',
                 options=[
                     {'label': 'All Sites', 'value': 'ALL'}
                 ] + [{'label': i, 'value': i} for i in spacex_df['Launch Site'].unique()],
                 value='ALL',
                 placeholder="Select a Launch Site",
                 searchable=True
                 ),
    html.Br(),

    # TASK 2: Add a pie chart to show the total successful launches count for all sites
    # If a specific launch site was selected, show the Success vs. Failed counts for the site
    html.Div(dcc.Graph(id='success-pie-chart')),
    html.Br(),

    html.P("Payload range (Kg):"),
    # TASK 3: Add a slider to select payload range
    dcc.RangeSlider(id='payload-slider',
                    min=0,
                    max=10000,
                    step=1000,
                    marks={0: '0', 2500: '2500', 5000: '5000', 7500: '7500', 10000: '10000'},
                    value=[min_payload, max_payload]),
    html.Br(),

    # TASK 4: Add a scatter chart to show the correlation between payload and launch success
    html.Div(dcc.Graph(id='success-payload-scatter-chart')),
])

# TASK 2:
# Add a callback function for `site-dropdown` as input, `success-pie-chart` as output
@app.callback(
    [Output(component_id='success-pie-chart', component_property='figure'),
     Output(component_id='success-payload-scatter-chart', component_property='figure')],
    [Input(component_id='site-dropdown', component_property='value'),
     Input(component_id="payload-slider", component_property="value")]
)
# TASK 4:
# Add a callback function for `site-dropdown` and `payload-slider` as inputs, `success-payload-scatter-chart` as output
def update_graph(selected_site, selected_range):
    if selected_site == 'ALL':
        filtered_df = spacex_df
    else:
        filtered_df = spacex_df[spacex_df['Launch Site'] == selected_site]

    success_counts = filtered_df[filtered_df['class'] == 1]['Launch Site'].value_counts()
    pie_chart_figure = {
        'data': [{
            'values': success_counts.values,
            'labels': success_counts.index,
            'type': 'pie',
            'hoverinfo': 'label+percent',
            'textinfo': 'value+percent'
        }],
        'layout': {'title': 'Total Successful Launches Count'}
    }

    bar_filtered_df = spacex_df[(spacex_df['Payload Mass (kg)'] >= selected_range[0]) &
                                (spacex_df['Payload Mass (kg)'] <= selected_range[1])]
    success_rate_df = bar_filtered_df[['class','Payload Mass (kg)']].groupby('Payload Mass (kg)').mean().reset_index()

    scatter_chart_figure = px.scatter(
        success_rate_df, x='Payload Mass (kg)', y='class',
        title=f'Success Rate of Payload Mass for Range {selected_range[0]} kg to {selected_range[1]} kg',
        labels={'class': 'Success Rate'},
        trendline='ols'
    )

    return pie_chart_figure, scatter_chart_figure

# Run the app
if __name__ == '__main__':
    app.run_server(debug=True)
